## Drone Notebook

This jupyter notebook is part of my 2020 summmer internship through Pacific University in Dr. Dawes' lab.

The objective of this notebook is to write and implement code to maneuver a Tello Edu drone, with the final goal of aligning two drones for optics experiments.

In [ ]:
import time
import socket
from tello import Tello



#### To Start
1. make sure tello.py in in same folder as this notebook
2. connect to Tello Drone wifi network

Some example code:

code modified from example on https://github.com/alecGraves/tello-udp

In [ ]:
# Declare a tello object
#this is a lot like turtles in 3D
drone = Tello()

# enable network commands on the drone
drone.send("command")

# takeoff
drone.send("takeoff")

start = time.time()
#initializes a start time

# wait 30 seconds
while time.time() - start < 30:
    # get battery info to prevent tello connection from timing-out
    # since tello auto-lands after 15 secs w/o a command
    drone.send("battery?")
    time.sleep(1)

# land the tello
drone.send("land")

In [ ]:
def real_time(drone,log=False):
    """This code is designed to put the user's tello drone into flight, then prompt the user 
    for commands in real time, as opposed to writing a long stream of commands in one code 
    cell
    
    parameters:
    drone: tello drone object
    optional boolean log: if true will return a log of commands used"""
    
    drone.send("command")
    #if the drone hasn't already been initialized to get commands this will do that
    print("Type tello command and hit enter. Type and send 'quit' to exit this program")
    
    running = True
    
    flight_log = []
    
    while running:
        #starts an infinite loop
        
        task = input('')
    #sends the message to the drone  
        drone.send(task)
        
        if task == 'land':
            running = False
            
        if log == True:
            flight_log.append(task)
            
    if log == True:
        return flight_log
        
        
                 

In [ ]:
real_time(drone)

## notes: this works!

In [ ]:
def polygon(drone,sides):
    """code to make drone take off fly in the polygon shape with specified sides
    
    drone: drone object
    sides: int number of sides of shape"""
    
    
    
    
    angle = 360/sides
    angle = int(angle)# has to be an int!
    
    drone.send('command')
    drone.send('takeoff')
     
    #creates a start time
    start = time.time()
    
    if time.time()% 5 == 0:
        # five second delay
        
        for i in range(0, sides):
            print(i)

            drone.send('forward 20')
            task = 'cw {0}'.format(angle)
            print(drone.send('battery?'))
            drone.send(task)
    drone.send('land')
        

In [ ]:
polygon(drone,5) 

## notes my drone will follow some of the commands but not all of them
# I might be able to fix this by adding a delay

In [ ]:
drone.send("battery?")

## checking for drift

So in the code below I will be checking for the drone's drift from a hovering point, with little to no wind/breeze during an indoor test with the windows closed

In [ ]:
def drift_test(drone, drift_time):
    """ This function returns data of how long it takes the tello drone to return to
    a starting location after drifting from that start"""

    drone.send('mon')
    drone.send('mdirection 0')
    drone.send('takeoff')
    #puts drone in air with mission pad detection on for pad below drone

    drone.send('go 0 0 0 m1')
    #places drone at center of mission pad 1


    start = time.time()
    #starts a timer
    
    if time.time() - drift_time < 0:
        drone.send("battery?")
        time.sleep(1)

    elif (time.time() - drift_time) >= 0 :
        reset = time.time()
        drone.send('go 0 0 0 m1')
        reset_time = (time.time() - reset)
    print( reset_time )
    return reset_time
    




In [ ]:
drone.send('command')
drift_test(drone,10)


### List of Commands:
#### (not all of them)

Setting commands:

* speed x

sets the speed between 10 and 100 ( the manual didn't specify units, but I'm assuming cm/s?)

ex: "speed 10"

Mission pad commands:

* mon

turns on mission pad detection

* moff 

turns off mission pad detection

* mdirection x 

sets mission pad detection direction (must turn on detection first)

0 = downward / 1 = forward / 2 = both

Motion Commands:

* takeoff (auto takeoff)
* land (auto landing)
* emergency ( stops all motors)
* up x (move up to 'x' cm above surface)
* down x (move down to 'x' cm above surface)
* forward/backward/left/right x (move 'x' cm in respective direction)
* cw/ccw x (rotate 'x' degrees in respetive direction)
